<a href="https://colab.research.google.com/github/Jorge2018/DataScience2022-2/blob/main/Script_caractBaseControl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import requests
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier


df=pd.read_csv(r'https://raw.githubusercontent.com/Jorge2018/RepositorioArchivos/main/bikeshare_train%20-%20bikeshare_train.csv')

df.head()


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 0:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 1:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 2:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 3:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 4:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


*Formateo de datos y eliminacion de cols*

In [2]:
df=df.drop(['casual','registered'],axis=1)
df['datetime'] = pd.to_datetime(df['datetime'])
df['monthName'] = df['datetime'].dt.month_name()
df['dayName'] = df['datetime'].dt.day_name()
df['hour_'] = df['datetime'].dt.hour
df['hour_']=df['hour_'].astype(object)
df=df.drop(['datetime','season'],axis=1)
df=df.assign(tempFahrenheit = lambda x: (9/5)*x['temp']+32)
df=df.assign(atempFahrenheit = lambda x: (9/5)*x['atemp']+32)
df['temp_variance']=df['atempFahrenheit']-df['tempFahrenheit']
df=df.drop(['atempFahrenheit','temp','atemp'],axis=1)
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   holiday         10886 non-null  int64  
 1   workingday      10886 non-null  int64  
 2   weather         10886 non-null  int64  
 3   humidity        10886 non-null  int64  
 4   windspeed       10886 non-null  float64
 5   count           10886 non-null  int64  
 6   monthName       10886 non-null  object 
 7   dayName         10886 non-null  object 
 8   hour_           10886 non-null  object 
 9   tempFahrenheit  10886 non-null  float64
 10  temp_variance   10886 non-null  float64
dtypes: float64(3), int64(5), object(3)
memory usage: 935.6+ KB


*Separacion de la columna objetivo*

In [3]:
y = df['count']
X = df.drop(columns=['count'])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

*Aplica sobre datos de entrenamiento y test Selector,Transformador,Imputer,Pipeline*

In [4]:
## **Instanciar selectores**
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

## Instanciar transformadores SimpleImputer, StandardScaler y OneHotEncoder
# Imputers
mean_imputer = SimpleImputer(strategy='mean')
freq_imputer = SimpleImputer(strategy='most_frequent')
# Scaler
scaler = StandardScaler()
# One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

## **Instanciar el pipeline**
numeric_pipe = make_pipeline(mean_imputer, scaler)
categorical_pipe = make_pipeline(freq_imputer, ohe)

## **Asignacion de tupla e instanciacion de ColumnTransformer**
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)
# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple)

## **Transformacion de datos**
# fit on train
preprocessor.fit(X_train,X_test)
# transform train and test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

*Revision del proceso de Selector,Transformador,Imputer*

In [5]:


print(np.isnan(X_train_processed).sum().sum(), 'valores perdidos en la data de entrenamiento')
print(np.isnan(X_test_processed).sum().sum(), 'valores perdidos en la data de prueba')
print('\n')
print('Todos los datos procesados del X_train_processed son del tipo:', X_train_processed.dtype)
print('Todos los datos procesados del X_test_processed son del tipo', X_test_processed.dtype)
print('\n')
print('shape de los datos de entrenamiento son', X_train_processed.shape)
print('\n')
print('shape de los datos de test son', X_test_processed.shape)



0 valores perdidos en la data de entrenamiento
0 valores perdidos en la data de prueba


Todos los datos procesados del X_train_processed son del tipo: float64
Todos los datos procesados del X_test_processed son del tipo float64


shape de los datos de entrenamiento son (8164, 50)


shape de los datos de test son (2722, 50)


*Crea instancias de los modelos y Revision de los hiperparametros*

In [6]:
dec_tree = DecisionTreeClassifier(random_state = 42, max_depth=30)
print('DecisionTree\n',dec_tree.get_params())
print('_______________________________________________')
bagreg = BaggingClassifier(random_state = 0)
print('Bagging\n',bagreg.get_params())
print('_______________________________________________')
knn = KNeighborsClassifier(n_neighbors=35)
print('KNeighbors\n',knn.get_params())

DecisionTree
 {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 30, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 42, 'splitter': 'best'}
_______________________________________________
Bagging
 {'base_estimator': None, 'bootstrap': True, 'bootstrap_features': False, 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 10, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}
_______________________________________________
KNeighbors
 {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 35, 'p': 2, 'weights': 'uniform'}


*Inserta valores en DF para su posterior analisis*

In [7]:


import time
import warnings
warnings.filterwarnings('ignore')

#lista=[10,30,50,70,100]
df_rev=pd.DataFrame()
start = time.time()
dec_tree.fit(X_train_processed, y_train)
dec_tree.predict(X_test_processed)
# calculen la exactitud de la clasificación
train_score = dec_tree.score(X_train_processed, y_train)
test_score= dec_tree.score(X_test_processed, y_test)
end = time.time()
t_proc=end-start
valor_ = pd.DataFrame({"accuracy_train":[train_score], "accuracy_test":[test_score],"time_proceso":[t_proc],"model":['DecisionTree'],"base":['Procesada']})
df_rev=df_rev.append(valor_)

start = time.time()
bagreg.fit(X_train_processed, y_train)
bagreg.predict(X_test_processed)
# calculen la exactitud de la clasificación
train_score = bagreg.score(X_train_processed, y_train)
test_score= bagreg.score(X_test_processed, y_test)
end = time.time()
t_proc=end-start
valor_ = pd.DataFrame({"accuracy_train":[train_score], "accuracy_test":[test_score],"time_proceso":[t_proc],"model":['Bagging'],"base":['Procesada']})
df_rev=df_rev.append(valor_)

start = time.time()
knn.fit(X_train_processed, y_train)
knn.predict(X_test_processed)
# calculen la exactitud de la clasificación
train_score = knn.score(X_train_processed, y_train)
test_score= knn.score(X_test_processed, y_test)
end = time.time()
t_proc=end-start
valor_ = pd.DataFrame({"accuracy_train":[train_score], "accuracy_test":[test_score],"time_proceso":[t_proc],"model":['KNeighbors'],"base":['Procesada']})
df_rev=df_rev.append(valor_)



In [8]:
df2=pd.read_csv(r'https://raw.githubusercontent.com/Jorge2018/RepositorioArchivos/main/bikeshare_train%20-%20bikeshare_train.csv')
df2=df2.drop(['casual','registered'],axis=1)
df2.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,2011-01-01 0:00:00,1,0,0,1,9.84,14.395,81,0.0,16
1,2011-01-01 1:00:00,1,0,0,1,9.02,13.635,80,0.0,40
2,2011-01-01 2:00:00,1,0,0,1,9.02,13.635,80,0.0,32
3,2011-01-01 3:00:00,1,0,0,1,9.84,14.395,75,0.0,13
4,2011-01-01 4:00:00,1,0,0,1,9.84,14.395,75,0.0,1


*Separacion de la columna objetivo*

In [9]:
y = df2['count']
X = df2.drop(columns=['count'])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

*Aplica sobre datos de entrenamiento y test Selector,Transformador,Imputer,Pipeline*

In [10]:
## **Instanciar selectores**
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

## Instanciar transformadores SimpleImputer, StandardScaler y OneHotEncoder
# Imputers
mean_imputer = SimpleImputer(strategy='mean')
freq_imputer = SimpleImputer(strategy='most_frequent')
# Scaler
scaler = StandardScaler()
# One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

## **Instanciar el pipeline**
numeric_pipe = make_pipeline(mean_imputer, scaler)
categorical_pipe = make_pipeline(freq_imputer, ohe)

## **Asignacion de tupla e instanciacion de ColumnTransformer**
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)
# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple)

## **Transformacion de datos**
# fit on train
preprocessor.fit(X_train,X_test)
# transform train and test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

*Revision del proceso de Selector,Transformador,Imputer*

In [11]:
print(np.isnan(X_train_processed).sum().sum(), 'valores perdidos en la data de entrenamiento')
print(np.isnan(X_test_processed).sum().sum(), 'valores perdidos en la data de prueba')
print('\n')
print('Todos los datos procesados del X_train_processed son del tipo:', X_train_processed.dtype)
print('Todos los datos procesados del X_test_processed son del tipo', X_test_processed.dtype)
print('\n')
print('shape de los datos de entrenamiento son', X_train_processed.shape)
print('\n')
print('shape de los datos de test son', X_test_processed.shape)

0 valores perdidos en la data de entrenamiento
0 valores perdidos en la data de prueba


Todos los datos procesados del X_train_processed son del tipo: float64
Todos los datos procesados del X_test_processed son del tipo float64


shape de los datos de entrenamiento son (8164, 8172)


shape de los datos de test son (2722, 8172)


*Inserta valores en DF para su posterior analisis*

In [12]:

start = time.time()
dec_tree.fit(X_train_processed, y_train)
dec_tree.predict(X_test_processed)
# calculen la exactitud de la clasificación
train_score = dec_tree.score(X_train_processed, y_train)
test_score= dec_tree.score(X_test_processed, y_test)
end = time.time()
t_proc=end-start
valor_ = pd.DataFrame({"accuracy_train":[train_score], "accuracy_test":[test_score],"time_proceso":[t_proc],"model":['DecisionTree'],"base":['Original']})
df_rev=df_rev.append(valor_)

start = time.time()
bagreg.fit(X_train_processed, y_train)
bagreg.predict(X_test_processed)
# calculen la exactitud de la clasificación
train_score = bagreg.score(X_train_processed, y_train)
test_score= bagreg.score(X_test_processed, y_test)
end = time.time()
t_proc=end-start
valor_ = pd.DataFrame({"accuracy_train":[train_score], "accuracy_test":[test_score],"time_proceso":[t_proc],"model":['Bagging'],"base":['Original']})
df_rev=df_rev.append(valor_)

start = time.time()
knn.fit(X_train_processed, y_train)
knn.predict(X_test_processed)
# calculen la exactitud de la clasificación
train_score = knn.score(X_train_processed, y_train)
test_score= knn.score(X_test_processed, y_test)
end = time.time()
t_proc=end-start
valor_ = pd.DataFrame({"accuracy_train":[train_score], "accuracy_test":[test_score],"time_proceso":[t_proc],"model":['KNeighbors'],"base":['Original']})
df_rev=df_rev.append(valor_)


In [13]:
df_rev

,accuracy_train,accuracy_test,time_proceso,model,base
0,0.802915,0.021675,0.539040,DecisionTree,Procesada
0,0.990446,0.022777,4.869359,Bagging,Procesada
0,0.071779,0.021675,3.690404,KNeighbors,Procesada
0,0.946717,0.009552,5.297495,DecisionTree,Original
0,0.991916,0.008450,298.080031,Bagging,Original
0,0.062347,0.011021,41.735897,KNeighbors,Original
